In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('/kaggle/input/flipkart-products/flipkart_com-ecommerce_sample.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df[['description', 'product_category_tree']]

In [ ]:
# Challenging
df.head()

In [ ]:
df['product_category_tree'] = df.product_category_tree.apply(lambda x: x.replace('[', '').replace(']', '').replace('"',''))

In [ ]:
x = df['product_category_tree'].apply(lambda x: x.split('>>'))

In [ ]:
x = x.apply(lambda x: x[0])

In [ ]:
df['category'] = x

In [ ]:
#df = df.drop('product_category_tree', axis=1)
df.head()

In [ ]:
counts = df.category.value_counts()

In [ ]:
most_frequent_category_names = counts[:10].index.tolist()
most_frequent_category_counts = counts[:10].values.tolist()
import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=most_frequent_category_names, y=most_frequent_category_counts))
fig.show()

In [ ]:
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
def count_words(matrix):
    words = []
    words_freq = []
    words_vals = []
    for desc in matrix:
        doc = nlp(desc)
        for token in doc:
            if not token.is_stop:
                words.append(doc)
    f = nltk.FreqDist(words)
    for x, v in f.most_common(10):
        words_freq.append(x)
        words_vals.append(v)
    return words, words_vals, words_freq

In [ ]:
words, words_vals, words_freq = count_words(df.category.values)

In [ ]:
from wordcloud import WordCloud

In [ ]:
im = WordCloud(width=640, height=800).generate(",".join([i.text for i in words]))

In [ ]:
import matplotlib.pyplot as plt
f, a = plt.subplots(figsize=(8, 12))
a.imshow(im, interpolation='nearest', aspect='auto')

In [ ]:
# Modelando
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
cv = CountVectorizer(stop_words='english')
tf = TfidfTransformer()

In [ ]:
df.dropna(inplace=True)
X = cv.fit_transform(df.description.values)

In [ ]:
X = tf.fit_transform(X)
y = df.category.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
classifier.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
param_grid = {'alpha':np.linspace(0, 1, 10), 'fit_prior':[True, False]}
grid = GridSearchCV(MultinomialNB(), param_grid=param_grid, cv=4)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
model = grid.best_estimator_
model.score(X_test, y_test)

In [ ]:
from textblob import TextBlob

def clasificador_español(model, cv, tf, producto='Zapatillas deportivas'):
    blob = TextBlob(producto)
    ingles = blob.translate(to='en')
    vect = cv.transform([ingles.raw])
    idf = tf.transform(vect)
    ingles = model.predict(idf)[0]
    blob = TextBlob(ingles)
    
    return blob.translate(to='es').raw

In [ ]:
clasificador_español(model, cv, tf)

In [ ]:
clasificador_español(model, cv, tf, 'Desatornillador Electrico')

In [ ]:
clasificador_español(model, cv, tf, 'Laptop Acer')

In [ ]:
clasificador_español(model, cv, tf,'Collar Swarovski Elements Corazón 14mm Cadena Plata')